this notebook is used to:
- sample data from attractor_data.csv
- save them as arrow files

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('../attractor_data.csv')

In [2]:
aizawax0 = eval(data[data['attractor'] == 'Aizawa_x_0']['values'].values[0])

In [35]:
def get_sample_parameter(seq_length = 512 + 64):
    """
    returns:
    - a random starting timestep t
    - a random step size between 60 - 120

    guarantees that t + 816 * step size < 324800
    324800 is the total timesteps we have for each attractor
    """

    import random
    max_index = 324800 - 1
    random_start_index = int(random.random() * 278820)

    # remaining = max_index - random_start_index
    max_step_size = 200
    min_step_size = 80

    random_step_size = min_step_size + int(random.random() * (max_step_size - min_step_size))

    return random_start_index, random_step_size

In [58]:
get_sample_parameter()

(2984, 98)

In [7]:
fp = 'dysts_dataset_generation_notes.csv'

In [64]:
# from Convert_to_arrow.py

from pathlib import Path
from typing import List, Union

import numpy as np
from gluonts.dataset.arrow import ArrowWriter


def convert_to_arrow(
    path: Union[str, Path],
    time_series: Union[List[np.ndarray], np.ndarray],
    compression: str = "lz4",
):
    """
    Store a given set of series into Arrow format at the specified path.

    Input data can be either a list of 1D numpy arrays, or a single 2D
    numpy array of shape (num_series, time_length).
    """
    assert isinstance(time_series, list) or (
        isinstance(time_series, np.ndarray) and
        time_series.ndim == 2
    )

    # Set an arbitrary start time
    start = np.datetime64("2000-01-01 00:00", "s")

    dataset = [
        {"start": start, "target": ts} for ts in time_series
    ]

    ArrowWriter(compression=compression).write_to_file(
        dataset,
        path=path,
    )

In [70]:
import os

seq_length = 512 + 64

lst = []
# sample 20 time series from each row in attractor_data.csv
for i in range(len(data)):
    series_name = data.iloc[i, 0]
    full_ts = eval(data.iloc[i, 1])
    for _ in range(20):
        # sample start index, time step
        start_index, step_size = get_sample_parameter()
        with open(fp, 'a') as csv:
            csv.write(f'{start_index},{step_size}\n')
        lst.append(full_ts[start_index:step_size * seq_length + start_index:step_size])
        # print(start_index, step_size)
convert_to_arrow(f'..\Data\dysts_arrow.arrow', lst)


In [71]:
len(data)

486